In [238]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import cross_validation
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import seaborn as sns

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   
from sklearn.grid_search import GridSearchCV   
from scipy.stats import chi2_contingency

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [256]:
def write_submission_file(estimator,name=""):
    estimator.fit(train[predictors],train[target])
    y_prob=estimator.predict_proba(test[predictors])[:, 1]
    output=pd.DataFrame([test[IDcol],y_prob],index=[IDcol,target]).T
    output.ID=output.ID.astype(int)
    if len(name):
        output.to_csv('submission_'+name+'.csv',index=False)
    else:
        output.to_csv('submission.csv',index=False)
        

# a way to gauge variability without reference to the value
        
def entropy(data,col,bins=10):
    if data[col].unique().shape[0]<bins:
        a,b=np.histogram(data[col],bins=np.sort(data[col].unique()))
        a=1.0*a/a.sum()
        return np.sum(-np.log(a)*a)
    else:
        range_col=data[col].unique()
        bins=np.linspace(range_col.min(),range_col.max(),bins)
        a,b=np.histogram(data[col],bins=bins)
        a=1.0*a/a.sum()
        a=a[a!=0]
        return np.sum(-np.log(a)*a)
    
    
def mutual_entropy(train_data,col,n_bins=10):
    if train_data[col].unique().shape[0]<n_bins:
        bins=np.sort(train_data[col].unique())
    else:
        range_col=train_data[col].unique()
        bins=np.linspace(range_col.min(),range_col.max(),n_bins)
    conf_matrix=np.zeros(shape=(2,bins.shape[0]))
    
    data0=train[train_data[target]==0]
    data1=train[train_data[target]==1]
    #p0=1.0*data0.shape[0]/train_data.shape[0]
    #p1=1.0*data1.shape[0]/train_data.shape[0]
    
    a0,b0=np.histogram(data0[col],bins=bins)
    a1,b1=np.histogram(data1[col],bins=bins)


    #a0=1.0*a0/a0.sum()*p0
    #a1=1.0*a1/a1.sum()*p1

    conf_matrix=[a0,a1]
    g, p, dof, expctd = chi2_contingency(conf_matrix, lambda_="log-likelihood")
    return g, p, dof, expctd

        
    
def analyze_col(col):
    plt.subplot(121)
    train[col].hist()
    plt.subplot(122)
    test[col].hist()
    plt.show()
    print col
    print np.sort(train[col].unique())
    print np.sort(test[col].unique())

In [212]:
# read in data and assign target and ID variables
train = pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
target = 'TARGET'
IDcol = 'ID'




In [213]:
train.shape

(76020, 371)

In [214]:
# remove constant columns (std = 0)
# notice here that we cannot take a peek into the test set for this purpose
# one may wonder how we know the variance is low in the test set also. 
# the idea is that we do NOT know but we know that the model that learns on the train set will not know
# what to do with these values in the test set so it doesn't matter
# check on the quality of the competition itself may be that there be not many deviations from this in the test set

remove_zero_var = []
for col in train.columns:
    if train[col].std() == 0:
        remove_zero_var.append(col)

train.drop(remove_zero_var, axis=1, inplace=True)
test.drop(remove_zero_var, axis=1, inplace=True)

# remove duplicated columns
remove_duplicate = []
cols = train.columns
for i in range(len(cols)-1):
    v = train[cols[i]].values
    for j in range(i+1,len(cols)):
        if np.array_equal(v,train[cols[j]].values):
            remove_duplicate.append(cols[j])

train.drop(remove_duplicate, axis=1, inplace=True)
test.drop(remove_duplicate, axis=1, inplace=True)



In [216]:
# choose predictors
predictors = [x for x in train.columns if x not in [target, IDcol]]

In [217]:
train.loc[train.var3==-999999,'var3']=0
test.loc[test.var3==-999999,'var3']=0

In [258]:
# check for entropy. default number of bins is a max of 1000 so max entropy is log(1000) = 6.9

remove=[]
for p in predictors:
    try:
        e=entropy(train,p)
        m=mutual_entropy(train,p)[1]
        print p,e,m
        if m<.1:
            remove.append(p)
    except:
        print "Some problem with "+p

var3 0.0429280133639 0.00210112440797
var15 1.36457820284 0.0
Some problem with imp_ent_var16_ult1
Some problem with imp_op_var39_comer_ult1
Some problem with imp_op_var39_comer_ult3
Some problem with imp_op_var40_comer_ult1
imp_op_var40_comer_ult3 0.014313800515 0.828811704399
Some problem with imp_op_var40_efect_ult1
Some problem with imp_op_var40_efect_ult3
Some problem with imp_op_var40_ult1
Some problem with imp_op_var41_comer_ult1
imp_op_var41_comer_ult3 0.0956882896207 0.467256309286
imp_op_var41_efect_ult1 0.0102491946744 0.00112665152757
Some problem with imp_op_var41_efect_ult3
Some problem with imp_op_var41_ult1
imp_op_var39_efect_ult1 0.0106398938856 0.000105089949824
Some problem with imp_op_var39_efect_ult3
Some problem with imp_op_var39_ult1
Some problem with imp_sal_var16_ult1
ind_var1_0 0.0 1.0
ind_var1 0.0 1.0
ind_var5_0 0.0 1.0
ind_var5 0.0 1.0
ind_var6_0 0.0 1.0
ind_var6 0.0 1.0
ind_var8_0 0.0 1.0
ind_var8 0.0 1.0
ind_var12_0 0.0 1.0
ind_var12 0.0 1.0
ind_var13_0 0.

In [259]:
remove

['var3',
 'var15',
 'imp_op_var41_efect_ult1',
 'imp_op_var39_efect_ult1',
 'num_var1',
 'num_var4',
 'num_var5_0',
 'num_var5',
 'num_var8_0',
 'num_var12_0',
 'num_var12',
 'num_var13_0',
 'num_var13_corto_0',
 'num_var13_corto',
 'num_var13_largo_0',
 'num_var13_largo',
 'num_var13',
 'num_var14_0',
 'num_var14',
 'num_var24_0',
 'num_var24',
 'num_var26_0',
 'num_var25_0',
 'num_op_var41_ult3',
 'num_op_var39_ult3',
 'num_var30',
 'num_var31',
 'num_var35',
 'num_var39_0',
 'num_var41_0',
 'num_var42_0',
 'num_var42',
 'saldo_var13_corto',
 'saldo_var30',
 'saldo_var40',
 'var36',
 'delta_num_aport_var13_1y3',
 'imp_aport_var13_hace3',
 'num_aport_var13_hace3',
 'num_med_var45_ult3',
 'num_meses_var5_ult3',
 'num_meses_var8_ult3',
 'num_meses_var12_ult3',
 'num_meses_var13_corto_ult3',
 'num_meses_var13_largo_ult3',
 'num_meses_var17_ult3',
 'num_meses_var39_vig_ult3',
 'num_meses_var44_ult3',
 'num_op_var40_efect_ult1',
 'num_op_var40_efect_ult3',
 'num_var45_hace2',
 'num_var45_u

In [261]:
train.drop(remove,axis=1,inplace=True)
test.drop(remove,axis=1,inplace=True)

In [11]:
#No categorical data


#categorical=[]
#numeral=[]
#for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
#    if train_name != test_name:
#        print "train and test name are not the same"
#        break
#    name=train_name
#    if train_series.dtype == 'O':
#        categorical.append(name)
#        train[name], tmp_indexer = pd.factorize(train[name])
#        test[name] = tmp_indexer.get_indexer(test[name])
#    else:
#        numeral.append(name)


In [263]:
train.shape,test.shape

((76020, 251), (75818, 250))

In [266]:
predictors = [x for x in train.columns if x not in [target, IDcol]]

In [264]:

# impute missing data

from sklearn.preprocessing import Imputer


imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
cols=train.columns
train=pd.DataFrame(imp.fit_transform(train),columns=cols)
cols=test.columns
test=pd.DataFrame(imp.fit_transform(test),columns=cols)



In [267]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.77842, std: 0.01206, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.77801, std: 0.01228, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.77814, std: 0.01206, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.77678, std: 0.01187, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.77608, std: 0.01179, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.77711, std: 0.01133, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.77010, std: 0.00960, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.77360, std: 0.01091, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.77317, std: 0.01123, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.76269, std: 0.00912, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.76770, std: 0.01045, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.76889, std: 0.01012, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 3, 'min_child_weight': 1

In [34]:
param_test1 = {
 'max_depth': [2,3,4],
 'min_child_weight':[4,5,6]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5,verbose=10)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  2.1min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  3.2min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  5.9min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  7.9min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed: 12.4min
[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed: 18.0min finished


[CV] max_depth=2, min_child_weight=4 .................................
[CV] max_depth=2, min_child_weight=4 .................................
[CV] max_depth=2, min_child_weight=4 .................................
[CV] max_depth=2, min_child_weight=4 .................................
[CV] ........ max_depth=2, min_child_weight=4, score=0.835947 - 1.1min[CV] ........ max_depth=2, min_child_weight=4, score=0.825416 - 1.1min[CV] ........ max_depth=2, min_child_weight=4, score=0.841164 - 1.1min[CV] ........ max_depth=2, min_child_weight=4, score=0.851997 - 1.1min



[CV] max_depth=2, min_child_weight=5 .................................[CV] max_depth=2, min_child_weight=5 .................................
[CV] max_depth=2, min_child_weight=5 .................................
[CV] max_depth=2, min_child_weight=4 .................................
[CV] ........ max_depth=2, min_child_weight=5, score=0.835926 - 1.0min[CV] ........ max_depth=2, min_child_weight=5, score=0.841485 - 1.0min
[CV] ...

([mean: 0.83805, std: 0.00864, params: {'max_depth': 2, 'min_child_weight': 4},
  mean: 0.83798, std: 0.00857, params: {'max_depth': 2, 'min_child_weight': 5},
  mean: 0.83795, std: 0.00814, params: {'max_depth': 2, 'min_child_weight': 6},
  mean: 0.83917, std: 0.00930, params: {'max_depth': 3, 'min_child_weight': 4},
  mean: 0.83928, std: 0.00895, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.83931, std: 0.00870, params: {'max_depth': 3, 'min_child_weight': 6},
  mean: 0.83868, std: 0.00859, params: {'max_depth': 4, 'min_child_weight': 4},
  mean: 0.83829, std: 0.00938, params: {'max_depth': 4, 'min_child_weight': 5},
  mean: 0.83891, std: 0.00948, params: {'max_depth': 4, 'min_child_weight': 6}],
 {'max_depth': 3, 'min_child_weight': 6},
 0.8393103908242544)

In [35]:
param_test1 = {
  'gamma':[i/10.0 for i in range(0,5)]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=3,
 min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5,verbose=10)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  3.1min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  4.7min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  7.7min
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed: 10.3min finished


[CV] gamma=0.0 .......................................................
[CV] gamma=0.0 .......................................................
[CV] gamma=0.0 .......................................................
[CV] gamma=0.0 .......................................................
[CV] .............................. gamma=0.0, score=0.834906 - 1.6min[CV] .............................. gamma=0.0, score=0.827175 - 1.6min[CV] .............................. gamma=0.0, score=0.843562 - 1.6min[CV] .............................. gamma=0.0, score=0.853144 - 1.6min



[CV] gamma=0.0 .......................................................[CV] gamma=0.1 .......................................................[CV] gamma=0.1 .......................................................
[CV] gamma=0.1 .......................................................

[CV] .............................. gamma=0.1, score=0.843566 - 1.5min
[CV] .............................. gamma=0.0, score=0.837766 - 1.5min[CV] ...

([mean: 0.83931, std: 0.00870, params: {'gamma': 0.0},
  mean: 0.83929, std: 0.00874, params: {'gamma': 0.1},
  mean: 0.83934, std: 0.00873, params: {'gamma': 0.2},
  mean: 0.83937, std: 0.00885, params: {'gamma': 0.3},
  mean: 0.83927, std: 0.00874, params: {'gamma': 0.4}],
 {'gamma': 0.3},
 0.8393700224931967)

In [36]:
param_test1 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=3,
 min_child_weight=6, gamma=.3, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5,verbose=10)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  2.7min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  3.9min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  6.3min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  7.8min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed: 11.7min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 14.6min
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed: 19.0min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed: 22.2min
[Parallel(n_jobs=4)]: Done  80 out of  80 | elapsed: 28.7min finished


[CV] subsample=0.6, colsample_bytree=0.6 .............................
[CV] subsample=0.6, colsample_bytree=0.6 .............................
[CV] subsample=0.6, colsample_bytree=0.6 .............................
[CV] subsample=0.6, colsample_bytree=0.6 .............................
[CV] .... subsample=0.6, colsample_bytree=0.6, score=0.833529 - 1.4min[CV] .... subsample=0.6, colsample_bytree=0.6, score=0.827155 - 1.4min[CV] .... subsample=0.6, colsample_bytree=0.6, score=0.842424 - 1.4min[CV] .... subsample=0.6, colsample_bytree=0.6, score=0.853612 - 1.4min



[CV] subsample=0.7, colsample_bytree=0.6 .............................[CV] subsample=0.7, colsample_bytree=0.6 .............................[CV] subsample=0.7, colsample_bytree=0.6 .............................[CV] subsample=0.6, colsample_bytree=0.6 .............................



[CV] .... subsample=0.7, colsample_bytree=0.6, score=0.834861 - 1.3min[CV] .... subsample=0.7, colsample_bytree=0.6, score=0.843236 - 1.3min[CV] ...

([mean: 0.83838, std: 0.00903, params: {'subsample': 0.6, 'colsample_bytree': 0.6},
  mean: 0.83942, std: 0.00817, params: {'subsample': 0.7, 'colsample_bytree': 0.6},
  mean: 0.83889, std: 0.00852, params: {'subsample': 0.8, 'colsample_bytree': 0.6},
  mean: 0.83909, std: 0.00855, params: {'subsample': 0.9, 'colsample_bytree': 0.6},
  mean: 0.83881, std: 0.00861, params: {'subsample': 0.6, 'colsample_bytree': 0.7},
  mean: 0.83910, std: 0.00815, params: {'subsample': 0.7, 'colsample_bytree': 0.7},
  mean: 0.83899, std: 0.00888, params: {'subsample': 0.8, 'colsample_bytree': 0.7},
  mean: 0.83891, std: 0.00811, params: {'subsample': 0.9, 'colsample_bytree': 0.7},
  mean: 0.83829, std: 0.00930, params: {'subsample': 0.6, 'colsample_bytree': 0.8},
  mean: 0.83937, std: 0.00880, params: {'subsample': 0.7, 'colsample_bytree': 0.8},
  mean: 0.83937, std: 0.00885, params: {'subsample': 0.8, 'colsample_bytree': 0.8},
  mean: 0.83883, std: 0.00878, params: {'subsample': 0.9, 'colsample_bytree'

[CV] .... subsample=0.7, colsample_bytree=0.9, score=0.825522 - 1.7min[CV] .... subsample=0.6, colsample_bytree=0.9, score=0.836142 - 1.7min



[CV] .... subsample=0.7, colsample_bytree=0.9, score=0.836281 - 1.7min[CV] .... subsample=0.7, colsample_bytree=0.9, score=0.842466 - 1.7min[CV] subsample=0.7, colsample_bytree=0.9 .............................[CV] subsample=0.8, colsample_bytree=0.9 .............................



[CV] .... subsample=0.8, colsample_bytree=0.9, score=0.826178 - 1.6min[CV] subsample=0.7, colsample_bytree=0.9 .............................
[CV] subsample=0.8, colsample_bytree=0.9 .............................[CV] .... subsample=0.7, colsample_bytree=0.9, score=0.836468 - 1.7min
[CV] .... subsample=0.7, colsample_bytree=0.9, score=0.853285 - 1.7min

[CV] subsample=0.8, colsample_bytree=0.9 .............................
[CV] .... subsample=0.8, colsample_bytree=0.9, score=0.835017 - 1.6min[CV] subsample=0.9, colsample_bytree=0.9 .............................
[CV] s

In [61]:
param_test1 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=3,
 min_child_weight=6, gamma=.3, subsample=0.7, colsample_bytree=0.6,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5,verbose=10)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  2.5min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  3.7min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  6.2min
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:  8.1min finished


[CV] reg_alpha=1e-05 .................................................
[CV] reg_alpha=1e-05 .................................................
[CV] reg_alpha=1e-05 .................................................
[CV] reg_alpha=1e-05 .................................................
[CV] ........................ reg_alpha=1e-05, score=0.834861 - 1.3min[CV] ........................ reg_alpha=1e-05, score=0.828782 - 1.2min[CV] ........................ reg_alpha=1e-05, score=0.843236 - 1.3min[CV] ........................ reg_alpha=1e-05, score=0.852885 - 1.2min



[CV] reg_alpha=0.01 ..................................................[CV] reg_alpha=1e-05 .................................................[CV] reg_alpha=0.01 ..................................................[CV] reg_alpha=0.01 ..................................................



[CV] ......................... reg_alpha=0.01, score=0.828282 - 1.2min[CV] ........................ reg_alpha=1e-05, score=0.837347 - 1.2min[CV] ...

([mean: 0.83942, std: 0.00817, params: {'reg_alpha': 1e-05},
  mean: 0.83902, std: 0.00824, params: {'reg_alpha': 0.01},
  mean: 0.83872, std: 0.00877, params: {'reg_alpha': 0.1},
  mean: 0.83940, std: 0.00857, params: {'reg_alpha': 1},
  mean: 0.82890, std: 0.00838, params: {'reg_alpha': 100}],
 {'reg_alpha': 1e-05},
 0.83942222127826338)

In [ ]:
param_test1 = {
    'learning_rate':[.01,.05, 0.1],
    'n_estimators':[100,140,190]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=3,
 min_child_weight=6, gamma=.3, subsample=0.7, colsample_bytree=0.6,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5,verbose=10)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


In [16]:
estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=3,
 min_child_weight=6, gamma=.3, subsample=0.7, colsample_bytree=0.6,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)

In [17]:
write_submission_file(estimator,"xgboost2")